# Distribution Functions
| Author  | Stanley A. Baronett |
|---------|---------------------|
| Created | 07/02/2021          |
| Updated | 08/02/2022          |

## [Probability Density Function (PDF)](https://en.wikipedia.org/wiki/Probability_density_function)
- A random variable $X$ has density $f_X$ if
\begin{align*}
    \mathrm{dP} &= f_{X}(x)\mathrm{d}x, \\
    \mathrm{P}(a \leq X \leq b) &= \int_{a}^{b} f_{X}(x) \mathrm{d}x.
\end{align*}
- Intuitively, one can think of $f_{X}(x)\,\mathrm{d}x$ as being the probability of $X$ falling within the infinitesimal interval $[x,x+\mathrm{d}x]$.
- PDFs are nonnegative everywhere, with an integral over the entire space equal to unity.
- A PDF can evaluate to greater than one, e.g., a uniform distribution on $\left[0, \tfrac{1}{2}\right]$ has $f_X(x) = 2$ for $0 \leq x \leq \tfrac{1}{2}$ and $f_X(x) = 0$ elsewhere.

## [Cumulative Distribution Functions (CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function) 
- CDFs are _non-decreasing_ and _right-continuous_, defined as $$F_{X}(x)=\mathrm{P}(X \leq x),$$ such that $$\lim _{x \rightarrow-\infty} F_{X}(x)=0, \quad \lim _{x \rightarrow \infty} F_{X}(x)=1$$
- Since the relationship between CDFs and PDFs is
    $$
    F_{X}(x)=\int_{-\infty}^{x} f_{X}(u) \mathrm{d}u,
    $$
  with
    $$
    f_{X}(x)=\frac{\mathrm{d}F_{X}(x)}{\mathrm{d}x},
    $$
  then
    $$
    \lim _{x \rightarrow \infty} F_{X}(x)=\int_{-\infty}^{\infty} f_{X}(u) \mathrm{d}u = 1,
    $$   

## Using CDF (Binless)

### np4 (cc-mod, 128$^2$)

In [3]:
# Collect .athdf outputs
athinput = athena_read.athinput('../nonlinear/BA/alt_runs/cc-mod/athinput.si')
outputs = sorted(list(Path('../nonlinear/BA/alt_runs/cc-mod/athdf').glob(athinput["job"]["problem_id"]+'.out2.*.athdf')))

In [ ]:
# Init sim constants and grid
nx1 = athinput['mesh']['nx1']              # num. radial zones
nx2 = athinput['mesh']['nx2']              # num. vertical zones
nx3 = athinput['mesh']['nx3']              # num. azimuthal zones
zones = nx1*nx2*nx3                        # total num. of zones
c_s = athinput['hydro']['iso_sound_speed'] # sound speed
Omega = athinput['problem']['omega']       # local Keplerian angular frequency
epsilon = athinput['problem']['epsilon']   # avg. dust/gas ρ-ratio in BG state
Np_tot = athinput['problem']['npx1']\
    *athinput['problem']['npx2']\
    *athinput['problem']['npx3']           # total number of particles
Np = Np_tot/nx1/nx2/nx3                    # theo avg num particles per cell
H = c_s / Omega                            # gas scale height
T = 2*np.pi/Omega                          # orbital period
data = athena_read.athdf(outputs[0])
xf, zf = data['x1f'] / H, data['x2f'] / H

#### References:
- [How to calculate and plot a Cumulative Distribution function with Matplotlib in Python](https://www.geeksforgeeks.org/how-to-calculate-and-plot-a-cumulative-distribution-function-with-matplotlib-in-python/)

In [ ]:
# Load data into memory
times, rhopxs = [], []

for output in outputs:
    data = athena_read.athdf(output)
    times.append(data['Time'] / T)
    temp = data['rhop'].flatten() / epsilon # flatten & convert
    temp = np.extract(temp>=0.1, temp)       # remove values < 0.1
    rhopxs.append(np.sort(temp))            # sort
    
# Construct CDFs
cdf = np.arange(rhopxs[0].size-1, -1, -1) / rhopxs[0].size
cdfs = np.full((len(outputs), cdf.size), cdf)

In [ ]:
# CPDD
fig, ax = plt.subplots(figsize=(6,5), dpi=100)
# ax.set_aspect('equal')
ax.set_title('CPDD: $t={:.2f}$ / $T$'.format(times[0]), size='x-large')
ax.set_xlabel(r'$\rho_p$ / $\langle \rho_p \rangle$', size='large')
ax.set_ylabel(r'P$(>\rho_p)$', size='large')
ax.loglog(rhopxs[0], cdfs[0])
ax.plot([1e-2, 1e3], [1e-1, 1e-1], '--', color='black')
ax.set_xlim(0.1, 1000)
ax.set_ylim(1e-5, 1)
ax.grid()

### np4 (256$^2$)

In [4]:
# Collect .athdf outputs
athinput = athena_read.athinput('../nonlinear/BA/athinput.si')
outputs = sorted(list(Path('../nonlinear/BA/athdf').glob(athinput["job"]["problem_id"]+'.out2.*.athdf')))

In [ ]:
# Init sim constants and grid
nx1 = athinput['mesh']['nx1']              # num. radial zones
nx2 = athinput['mesh']['nx2']              # num. vertical zones
nx3 = athinput['mesh']['nx3']              # num. azimuthal zones
zones = nx1*nx2*nx3                        # total num. of zones
c_s = athinput['hydro']['iso_sound_speed'] # sound speed
Omega = athinput['problem']['omega']       # local Keplerian angular frequency
tau_s = athinput['particles']['taus0']*Omega # dimensionless stopping time
epsilon = athinput['problem']['epsilon']   # avg. dust/gas ρ-ratio in BG state
Np_tot = athinput['problem']['npx1']\
    *athinput['problem']['npx2']\
    *athinput['problem']['npx3']           # total number of particles
Np = Np_tot/nx1/nx2/nx3                    # theo avg num particles per cell
H = c_s / Omega                            # gas scale height
T = 2*np.pi/Omega                          # orbital period

data = athena_read.athdf(outputs[0])
xf, zf = data['x1f'] / H, data['x2f'] / H

In [ ]:
# Load data into memory
times, rhopxs = [], []

for output in outputs:
    data = athena_read.athdf(output)
    times.append(data['Time'] / T)
    temp = data['rhop'].flatten() / epsilon # flatten & convert
    temp = np.extract(temp>=0.1, temp)      # remove values < 0.1
    rhopxs.append(np.sort(temp))            # sort
    
# Construct CDFs
# cdf = np.arange(rhopxs[0].size-1, -1, -1) / rhopxs[0].size
cdf = np.linspace(1, 0, rhopxs[0].size, endpoint=False)

In [ ]:
rhopx1 = rhopxs[0]

In [ ]:
# CPDD
fig, ax = plt.subplots(figsize=(8,6), dpi=80)
# ax.set_aspect('equal')
ax.set_title('CPDD: $t={:.2f}$ / $T$'.format(times[0]), size='x-large')
ax.set_xlabel(r'$\rho_p$ / $\langle \rho_p \rangle$', size='large')
ax.set_ylabel(r'P$(>\rho_p)$', size='large')
ax.loglog(rhopxs[0], cdf,
          label=r'$\tau_s={:.1f},\,\epsilon={:.1f}$'.format(tau_s, epsilon))
ax.plot([1e-2, 1e3], [1e-1, 1e-1], '--', color='black')
ax.set_xlim(0.1, 1000)
ax.set_ylim(1e-5, 1)
ax.legend()
ax.grid()

## Using Historgram (Bins)

In [ ]:
# Load data into memory
bins = np.logspace(-1, 3, 1000)
# bins = np.linspace(0.1, 1e3, 1000)
times, rhopxs = [], []

for output in outputs:
    data = athena_read.athdf(output)
    times.append(data['Time'] / T)
    rhopxs.append(np.sort(data['rhop'].flatten()) / epsilon) # flat & sorted

In [ ]:
# CPDD
fig, ax = plt.subplots(figsize=(6,5), dpi=100)
# ax.set_aspect('equal')
ax.set_title('CPDD: $t={:.2f}$ / $T$'.format(times[0]), size='x-large')
ax.set_xlabel(r'$\rho_p$ / $\langle \rho_p \rangle$', size='large')
ax.set_ylabel(r'$P(>\rho_p)$', size='large')
ax.hist(rhopxs[0], bins, cumulative=-1, density=True,
        histtype='bar',  rwidth=0.7, 
        label=r'$\tau_s={:.1f},\,\epsilon={:.1f}$'.format(tau_s, epsilon))
ax.plot([1e-2, 1e3], [1e-1, 1e-1], '--', color='black')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.1, 1000)
ax.set_ylim(1e-5, 1)
ax.legend()
ax.grid()

## CDF vs. Histogram Comparison

In [ ]:
# CPDD
fig, ax = plt.subplots(figsize=(6,5), dpi=100)
# ax.set_aspect('equal')
ax.set_title(r'CPDD: $t={:.2f}$ / $T \qquad (\tau_s={:.1f},\,\epsilon={:.1f})$'\
             .format(times[0], tau_s, epsilon), size='x-large')
ax.set_xlabel(r'$\rho_p$ / $\langle \rho_p \rangle$', size='large')
ax.set_ylabel(r'$P(>\rho_p)$', size='large')
ax.loglog(rhopx1, cdf, label='From CDF')
ax.hist(rhopxs[0], bins, cumulative=-1, density=True,
        histtype='bar', label='Histogram', rwidth=0.7)
ax.plot([1e-2, 1e3], [1e-1, 1e-1], '--', color='black')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0.1, 1000)
ax.set_ylim(1e-5, 1)
ax.legend()
ax.grid()

## CPDDs (w/ SD in log space)

A **standard deviation** _in log space_ gives equal lengths on either side of the average on a log-scale axis.

$$ \sigma = \exp({\textrm{SD}[\ln X]}) $$
$$ [\sigma^{-1}\mu,\,\sigma\mu] $$

_Source_: [CC's Slack messages](https://unlv-diskandplanet.slack.com/archives/C01B0NBE3CY/p1627591911000500?thread_ts=1627162756.000400&cid=C01B0NBE3CY)

## Tasks
- [x] Undestand probability density function (PDF)
- [x] Why does combining PDF and CDF always normalize (max 1)?
- [x] Read §§ 4, 4.1, 4.2
- [x] No sum needed; raw values from each zone/cell
- [x] Rewrite CPDD algo using histogram func
  - [x] Consider appropriate bin size (cf. JY07)
- [x] Check 256$^2$, `np25` results against w/ JY07
- [x] Debug script if unsure of results
- [x] Try modifying original CDF method by cutting off < 0.1 counts (should renormalize)
- [x] Script to avg CPDD over saturation outputs
- [x] Read & plot averaged CPDD_np4.txt results from NAS
- [x] Set up cpdd.py & cpdd.pbs for np9 - np25
- [x] CPDD comparison plot of each run (cf. Fig. 11)
- [x] Rewrite script w/o cutting (see comments in cell below)
- [x] Compute std. dev. in log space (`cpdd3.py`)
- [ ] Recompute log. std. dev. for np4 - np16 runs (replace above)
- (Old)
  - [ ] ? Measure density contrast/turbulent properties
  - [ ] ? Morphology difference, but growth properties

In [ ]:
# Chao-Chin:
# NO CUTTING
# - Sort each snapshot in increasing order
# - Determine min/max values over snapshot range,
#   for largest cell value, second largest, so on
#   (i.e., copy line 62 for a min and max series)
# - plot w/ 3 lines: min, avg, max
# - compute std_dev of values over time (plot w/
#   horizontal err. bars for each data pt., i.e.
#   "cell")